# Исследование объявлений о продаже квартир

**<br/> Нужно научиться определять рыночную стоимость объектов недвижимости.**
<br/>**Задача — установить параметры для построения автоматизированной системы: она отследит аномалии и мошенническую деятельность.** 

<br/>**Что делаем**

<br/>**Предобработка**

<br/>**Добавляем в таблицу:**
<br/>цену кв. метра
<br/>день недели месяц и год рубликации объявления
<br/>Этаж квартиры(первый последний, другой)
<br/>Отношение жилой и общей площади
<br/>Отношение площади кухни к общей
<br/>Расстояние до центра в км(Int)
<br/>***Строим гистограмму для:***
<br/>площадь
<br/>Цена
<br/>Число комнат
<br/>Высота потолков
<br/>Время продажи квартиры
<br/>**Для Время продажи** - посчитать медиану и среднее, убрать редкие и выбивающиеся значения
<br/>**Построить зависимости:**
<br/>цена - площадь
<br/>цена - число комнат
<br/>цена - удаленность от центра
<br/>цена - этажность
<br/>цена - дата размещения
<br/>**Строим сводную таблицу топ 10(по кол. объявления) населенных пунктов с самой высокой и самой низкой стоимостью жилья**
<br/>Насеоенный пункт - количество объявлений - стоимость квадратного метра.
<br/>**Определяем какая область входит в центр:**
<br/>считаем среднюю цену для каждого километра от цента в СПб
<br/>сторим график зависимости
<br/>**Строим зависимости для квартир в центре**
<br/>см общие

### Шаг 1. Откройте файл с данными и изучите общую информацию.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer
from collections import Counter
import numpy as np
import math as mth
from datetime import datetime

ModuleNotFoundError: No module named 'pymystem3'

In [ ]:
#estate.head(20)
flat = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
#flat.head(20)

In [ ]:
flat.info()

In [ ]:
flat.duplicated().sum()

In [ ]:
flat.isnull().sum()

 describe(). У последней есть классный аргумент include. Например include = ['object']

### Вывод

**first_day_exposition** – формат
<br/>**ceiling_height** – отсутствие значений: скорее всего зачастую продавец не считает этот параметр важным и влияющим на стоимость. Только 3х метровые потолки и выше, по моему мнению, могут влиять на стоимость, потому что за такие потолки уже можно просить доп. деньги (это комфорт, и в такой кв. можно сделать мансардный этаж)  взять стандартные потолки 2.70 или 2.50 или нулями заменить. Есть странности в максимаьных значениях
<br/>**living_area , kitchen_area** – выжные параметры влияющие на стоимость. 
Сколько отсутствующих значений в обоих параметрах – удалить
Заамеить отсутствующие значения в living_area  средним для сегмента по числу комнат и стоимости 
<br/>**is_apartment** – никто не знает что это значит, кроме тех кто продает апартаменты, будем считать, что все NaNы – это false
<br/>**balcony** – nan заменим на 0  -балкон если есть его надо указывать это влияет на стоимость, вряд ли кто то поленился указать.  
<br/>**airports_nearest** – заменяем на 0
<br/>**locality_name** - удалим
<br/>**cityCenters_nearest** – берем расстояния от центра для населеннго пункта
<br/>**parks_around3000** – парки оставляем или заменяем на 0
<br/>**parks_nearest** – тоже самое, можно поимскать парки по населенному пункту
<br/>**ponds_around3000** – наплевать на прууды
<br/>**days_exposition** – нет смысла заменять эти значения, будем оценивать имеющиеся данные
<br/>**типы данных поменять**
<br/>дату
<br/>balcony -to int

<br/>floors_total – to int
<br/>is_apartment – to bool
<br/>days_exposition to int

**Дубликатов нет**


### Шаг 2. Предобработка данных

In [ ]:
#высота потолков
#flat[flat['ceiling_height'] > 5].sort_values('ceiling_height')

In [ ]:
#100 метровые потолки удалим.
flat.drop([6,22869], inplace = True)

In [ ]:
# заменим совсем уж сказочные значения на стандартные по региону
#def fix_ceiling_height(row):
   # if row > 4:
     #   return 2.5
   # elif row >= 25:
     #   return row / 10
   # else:
     #   return row
#flat['ceiling_height'] = flat['ceiling_height'].apply(fix_ceiling_height)
#flat.iloc[21823]
#flat[flat['ceiling_height'] > 4].sort_values('ceiling_height')

In [ ]:
def fix_ceiling_height(row):
    if row < 4:
        return row
    if row > 25:
        return row / 10
    return 2.5               
       
flat['ceiling_height'] = flat['ceiling_height'].apply(fix_ceiling_height)
#flat.iloc[21823]
flat[flat['ceiling_height'] > 4].sort_values('ceiling_height')

https://medium.com/lemon-code/guard-clauses-3bc0cd96a2d3

In [ ]:
#заменим пропуски на стандартную высоту потолков 2,5
flat['ceiling_height'] = flat['ceiling_height'].fillna(2.5)
flat['ceiling_height'].isnull().sum()

In [ ]:
#балконы на 0
flat['balcony'] = flat['balcony'].fillna(0)
#заменим тип данных
flat['balcony'] = flat['balcony'].astype('int', errors='ignore')
#flat.info()

In [ ]:
mean_rooms_area = pd.pivot_table(flat, index = 'rooms', values = 'living_area').sort_values('rooms')
mean_rooms_area.head(20)

In [ ]:
#возьмем среднюю площадь квартир для каждого колиества комнат и заменим наши пропуски 
flat['living_area'] = flat.groupby('rooms')['living_area'].transform(lambda x: x.fillna(x.mean()))
#flat.iloc[14602]

In [ ]:
#flat.sort_values('locality_name')['locality_name'].value_counts()

In [ ]:
flat[flat['locality_name'].isnull()]

**49 строк с неизвестным городом.
У нас в строках с пропусками в названии насеоенного пункта есть расстояние от центра.
Пусть все кто в пределах 20 км  - будут считаться Санкт Петербургом - так пропусков станет поменьше**

In [ ]:
flat.loc[flat['cityCenters_nearest'] < 20000, 'locality_name'] = flat.loc[flat['cityCenters_nearest'] < 20000, 'locality_name'].fillna('Санкт-Петербург')
#flat.head(20)

In [ ]:
flat['locality_name'].isnull().sum()

In [ ]:
flat.sort_values('locality_name')['locality_name'].value_counts()
#вроде все на месте

In [ ]:
#измним формат даты, уберем лишнее
flat['first_day_exposition'] = pd.to_datetime(flat['first_day_exposition'], format = '%Y-%m-%d')
#flat['first_day_exposition'].head(5)
#balcony to int
flat['balcony'] = flat['balcony'].astype('int', errors='ignore')
# floors_total- to int
flat['floors_total'] = flat['floors_total'].astype('int', errors='ignore')
#is_apartment – to bool
flat['is_apartment'] = flat['is_apartment'].astype('bool', errors='ignore')
#days_exposition to int
flat['days_exposition'] = flat['days_exposition'].astype('int', errors='ignore')
#'last_price' to int
flat['last_price'] = flat['last_price'].astype('int', errors='ignore')
flat['total_area'] = flat['total_area'].astype('int', errors='ignore').round(2)

flat.info()

**Выводы по предобработке:** 

<br/>**first_day_exposition** – поправил формат
<br/>**ceiling_height** – заменили отсутствующие и сказочные значения на стандартную высоту потолков в России 2.5., потому что если продавец поленился измерить высту потолков или попросту не посчитал этот параметр важным, значит он не влияет на стоимость квартиры и, следовательно, является стандартным по России. Кроме того, были 100 метровые потолки - удалии(1 значение).Странные значения от 4 метров и выше заменил на стандартные 2,5, 25 и 27 метров - здесь можно сделать вывод, что это артефакт и в этих цифрах просто не проставили запятую в нужном месте - исправил (19 значения). В дальнейшем посмотрим не осталось ли выбивающихся значений. Думаю основная причина пропусков и неверных диких значений  -  это недобросовестное заполнение этих пунктов пользователем.
<br/>**living_area (1903 пропуска) - для этих строк пвзял среднюю жилую площадь для n - комнатной квартиры. 


<br/>**is_apartment** – никто не знает что это значит, кроме тех кто продает апартаменты, будем считать, что все NaNы – это false, потому что, человек, когдато купивший апарты должен был разобраться что это и, соответственно, при продаже обратитть внимание на этот пункт. Есть еще вариант, что данные начали собирать до того как появилась такая категория жилья. В любом случае восстановить эти значения по имеющимся данным не получится (можно было оставить пропуски)
<br/>**balcony** – nan заменим на 0  -балкон если есть его надо указывать это влияет на стоимость, вряд ли кто то поленился указать.  
<br/>**airports_nearest** – оставил
<br/>**locality_name** - по расстоянию от центра присвоили некоторым пропущенным значениям название "Санкт-Петербург" 
<br/>**cityCenters_nearest, parks_around3000, parks_nearest, ponds_around3000, days_exposition** – оставил пропуски
<br/>**типы данных поменять**

Произвел замену типов данных для красоты и удобства

**Дубликатов нет**

### Шаг 3. Посчитайте и добавьте в таблицу

In [ ]:
#Добавляем в таблицу:
#цену кв. метра
#день недели месяц и год рубликации объявления
flat['price_per_sqmeters'] = (flat['last_price'] / flat['total_area']).astype('int')
flat['post_weekday'] = flat['first_day_exposition'].dt.weekday
flat['post_month'] = flat['first_day_exposition'].dt.month
flat['post_year'] = flat['first_day_exposition'].dt.year
#Этаж квартиры(первый последний, другой)
def floor_category(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['floors_total']:
        return 'последний'
    return 'другой'
flat['floor_category'] = flat.apply(floor_category, axis=1)
#Отношение жилой и общей площади
flat['useful_area'] = flat['living_area'] / flat['total_area']
#Отношение площади кухни к общей
flat['kitchen_ratio_total_area'] = flat['kitchen_area'] / flat['total_area']
flat.head()

In [ ]:
flat['ceiling_height'].value_counts()


## ВЫВОД:
**Произвел рассчеты параметров, необходимых для анализа данных, и добавил их в таблицу**

## Шаг 4. Проведите исследовательский анализ данных и выполните инструкции:

***4.1 Изучите следующие параметры: площадь, цeна, число комнат, высота потолков. Постройте гистортщграммы для каждого параметра***

In [ ]:
#изучем параметры
flat[['total_area', 'rooms', 'ceiling_height']].describe()

In [ ]:
#гистограмму для total_area
flat.plot(y='total_area', kind='hist', bins=100, grid=True, figsize=(20,5), range=(0,300))
plt.xlabel("total_area")
plt.ylabel("Frequency")

In [ ]:
flat['last_price'] = flat['last_price'] / 1000
flat['last_price'].describe()

In [ ]:
#гистограмма для last_price
flat.plot(y='last_price', kind='hist', bins=100, grid=True, range=(400,13000), figsize=(10,5))
plt.xlabel('last_price')

In [ ]:
#гистограмма для rooms
flat.plot(y='rooms', kind='hist', bins=30, grid=True, range=(0,20), figsize=(10,5))
plt.xlabel('rooms')

In [ ]:
#гистограмма для ceiling_height
flat.plot(y='ceiling_height', kind='hist', bins=10, grid=True, range=(2,5), figsize=(10,5))
plt.xlabel("ceiling_height")

### Выводы по 4.1:
**В выборку попали площади попали от 12 до 900 кв. м. Среднее значение 60. Большая часть продаваемых квартир - это квартиры 40-60 кв. м
Стоимость активно продаваемого жилья 3,4 - 6,8 млн. руб. в основном это 1,2,3х комнатные квартиры, преобладают 1,2х комнатные. Высота потолков в основном 2,5 - 2,7 м., есть 3х метровые**  


## 4.2.
***Изучите время продажи квартиры. Постройте гистограмму. Посчитайте среднее и медиану. Сделать выводы.***

In [ ]:
flat['days_exposition'].describe()

In [ ]:
**Максимум квартиру продавали 4 с лишним года. В среднем квартира продается за пол года. Ходя медиана около 4 месяцев.**

In [ ]:
flat.plot(y = 'days_exposition', kind = 'hist', bins = 30, grid = True, figsize=(10,5), range = (1,1600))
plt.xlabel("days_exposition")



In [ ]:
#посмотрим выбросы
flat.boxplot(column='days_exposition')

In [ ]:
#построим гистограмму без выбросов
flat.plot(y = 'days_exposition', kind = 'hist', bins = 100, grid = True, figsize=(10,5), range = (1,500))
plt.xlabel('days_exposition')

In [ ]:
#посмотрим, что за пики
flat.plot(y = 'days_exposition', kind = 'hist', bins = 70, grid = True, figsize=(10,5), range = (40,70))
plt.xlabel("total_area")

## Выводы по 4.2:
**в основной массе квартиры продаются в первые 90 дней. Максимум квартиру продавали 4 с лишним года . В среднем квартира продается за пол года. Медианное значение около 4 месяцев. Странные пики на 45 и 60 могут быть системными (например снятие обьявление с публикации через опр период)**

## 4.3 
**Изучите, зависит ли цена от площади, числа комнат, удаленности от центра. <br/>Цена от этажа, на кот. расположена кв. <br/>Цена от даты размещения: дня, месяца, года**

In [ ]:
#total_area
total_area_relation = flat.pivot_table(index = 'total_area', values = 'price_per_sqmeters', aggfunc = ['mean', 'median'])
total_area_relation.columns = ['mean', 'median']
total_area_relation.sort_values('mean', ascending=False)

In [ ]:
flat['total_area'].corr(flat['price_per_sqmeters'])
#с увеличением площади цена растет, но незначительно!

In [ ]:
flat.plot(x='total_area', y='price_per_sqmeters', kind='scatter') 

In [ ]:
(flat
         .query('price_per_sqmeters < 250000 and total_area < 200')
         #.plot(x='total_area', y='price_per_sqmeters', kind='hexbin', gridsize=15, figsize=(10,5), sharex=False, grid=True)
         .plot(x='total_area', y='price_per_sqmeters', kind='scatter',figsize=(20,10), alpha=0.3)
)

### ВЫВОД : 
**с жильем до 100 м кв. цена колеблется примерно от 25000 до 200000 т. С увеличением площади жилья прослеживается незначительный рост цены за кв. метр. С увеличением площади жилья разброс цен сужается - завышать стоимость жилья на значительные площади сложнее, поэтому цена за кв метр стремится к средним значениям**

In [ ]:
#число комнат
rooms_relation = flat.pivot_table(index = 'rooms', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
rooms_relation.columns = ['mean', 'count', 'median']
rooms_relation.sort_values('rooms')

In [ ]:
(rooms_relation
                 .query('median < 300000')
                 .plot(y='median', style='-')
)
#цена растет до 7 комна

In [ ]:
flat['rooms'].corr(flat['last_price'])

In [ ]:
rooms_relation = flat.pivot_table(index = 'rooms', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
rooms_relation.columns = ['mean', 'count', 'median']
rooms_relation.sort_values('rooms')

In [ ]:
(rooms_relation
                 .query('median < 300000')
                 .plot(y='median', style='o')
)

## Вывод:
**Стоимость квартиры логично повышается с увеличением числа комнат. Но квартиры от 7 до 10 комнат судя по графику стоят почти одинаково. Стоимость квадратного метра жилья снижается до 3х комнатных квартир. Судя по графику самая низкая стоимость квадратного метра жилья у 3х комнатных квартир. от 3х комнат и выше стоимость квадратного метра внова растет.** 

### Удаленность от центра - цена

In [ ]:
center_relation = flat.pivot_table(index = 'last_price', values = 'cityCenters_nearest')
#rooms_relation.columns = ['mean', 'count', 'median']
center_relation.sort_values('last_price')

In [ ]:
(center_relation
                 .query('last_price < 150000 and cityCenters_nearest < 30000')
                 .plot(style='o', figsize=(10,6))
)

### ВЫВОД:
**ближе к центру города определенно увеличивается сегмент дорогого жилья, но нижняя граница цен растет незначительно, т.е в центре есть некий сегмент недвижимости по стандартным для города ценам - вопрос, что это за жилье и какого уровня.**

### Цена от этажа, на кот. расположена кв.

In [ ]:
floor_relation = flat.pivot_table(index = 'floor_category', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
floor_relation.columns = ['mean', 'count', 'median']
floor_relation.sort_values('mean')

In [ ]:
(floor_relation
                 #.query('median < 6000 and floor < 12 and count > 200')
                 .plot(y='median', style='-', figsize=(10,6))
)

### ВЫВОД:
***самый дешевый первый этаж, посдледний деше стоит дешевле чем другие, но дороже чем первый*** 

## Цена от даты размещения: дня, месяца, года

In [ ]:
post_year_relation = flat.pivot_table(index = 'post_year', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
post_year_relation.columns = ['mean', 'count', 'median']
post_year_relation.sort_values('post_year')
(post_year_relation
                 #.query('median < 6000 and floor < 12 and count > 200')
                 .plot(y='mean', kind='bar', figsize=(10,6))
)

**ВЫВОД: рекордные цены по выборке в 2014 году - скорее всего это было спровоцированно валютным кризисом 2014-2015 г, далее все пошло на спад и к 2016 цена нормализовалась. в 2018 - заметное снижение** 

In [ ]:
post_month_relation = flat.pivot_table(index = 'post_month', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
post_month_relation.columns = ['mean', 'count', 'median']
post_month_relation.sort_values('post_month')
(post_month_relation
                 #.query('median < 6000 and floor < 12 and count > 200')
                 .plot(y='mean', kind='bar', figsize=(10,6))
)

### ВЫВОД
***цена циклически снижается - повышается по кварталам, начиная с начала года. Пик в сентябре нарушает картину - возможно, какие то события подхлестнули спрос!***

In [ ]:
day_relation = flat.pivot_table(index = 'post_weekday', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
day_relation.columns = ['mean', 'count', 'median']
day_relation.sort_values('post_weekday')

In [ ]:
(day_relation
                 #.query('median < 6000 and floor < 12 and count > 200')
                 .plot(y='mean', kind='bar', figsize=(10,6))
)

### ВЫВОД:
***зависимость не прослеживается. Единственное, что можно увидеть - повышение цены в субботу***

### 5. ***Выбрать 10 населенных пунктов с наибольшим числом обьявления. найди для них среднюю цену. найти населенные пункты с самой низкой и самой высокой стоимостью жилья***

In [ ]:
top_10 = flat['locality_name'].value_counts().index.tolist()[:10]
high_low_top_price = [flat[flat['locality_name'] == city]['last_price'].median() for city in top_10]
frame = {'Город': top_10, 'цена': high_low_top_price}
df = pd.DataFrame(frame).sort_values('цена', ascending=False)
df

### ВЫВОД:
**Самые дорогие квартиры в Санкт-Петербурге и Пушкине, самые дшевые в Выборге!**

### График для расстояния от центра

In [ ]:
spb_flats = flat.query('locality_name == "Санкт-Петербург"')

In [ ]:
spb_flats.loc[:, 'cityCenters_nearest_km'] = (spb_flats.loc[:, 'cityCenters_nearest'] / 1000).round()
spb_flats.head(10)

In [ ]:
km_relation = spb_flats.pivot_table(index = 'cityCenters_nearest_km', values = 'last_price', aggfunc = ['mean', 'count', 'median'])
km_relation.columns = ['mean', 'count', 'median']
km_relation.sort_values('cityCenters_nearest_km')

In [ ]:
(km_relation
                 .query('cityCenters_nearest_km < 100')
                 .plot(y='median', style='-', figsize=(10,6))
)

### ВЫВОД: 
***график показал что цена резко меняется в сторону увеличения на 1,3 км - определим это как самый центр. Общий рост график показывает на 9 км -этот радиус и будем считать центром города. Есть пики на 20, и даже 30 км - скорое всего они относятся к неким элитным районам, пользующимся спросом вне зависимости от удаленности от центра.*** 

### Смотрим на квартиры в центре

In [ ]:
spbCenter_data = spb_flats.query('cityCenters_nearest_km < 9')
spbCenter_data

In [ ]:
spbCenter_data[['last_price', 'total_area', 'rooms', 'ceiling_height']].describe()

In [ ]:
flat[['last_price', 'total_area', 'rooms', 'ceiling_height']].describe()

In [ ]:
flat.info()

In [ ]:
#посмотррим завиимость стоимости кв метра от этажа в центре.
spbCenter_floor_relation = spbCenter_data.pivot_table(index = 'floor_category', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
spbCenter_floor_relation.columns = ['mean', 'count', 'median']
spbCenter_floor_relation.plot(y = 'median')
spbCenter_floor_relation

**ВЫВОД: В центре зависимость стоимости квадратного метра жилья от этажности сохраняется такая же как и по остальным районам первый - самый дешевый, последний подороже, чемпервый, остальные - в пользуются спросом и стоят дороже.**

In [ ]:
#смотрим зависимость стоимости жилья в центре от количества комнат
spbCenter_rooms_relation = spbCenter_data.pivot_table(index = 'rooms', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
spbCenter_rooms_relation.columns = ['mean', 'count', 'median']
spbCenter_rooms_relation.plot(y = 'median')
spbCenter_rooms_relation

In [ ]:
spbCenter_data['rooms'].corr(spbCenter_data['price_per_sqmeters'])

**ВЫВОД: в центре зависимость цены от количества комнат незначительная.**

In [ ]:
#смотрим зависимость по удаленности от центра, хотя мы уже в центре! 
spbCenter_data.plot(kind = 'scatter', y = 'price_per_sqmeters', x = 'cityCenters_nearest', alpha=0.3 )
spbCenter_data['cityCenters_nearest'].corr(spbCenter_data['price_per_sqmeters'])


**ВЫВОД: изменений цены по всему центру не наблюдается, все ровненько**

In [ ]:
#зависимость день недели стоимость кв метра в центре
spbCenter_pw_relation = spbCenter_data.pivot_table(index = 'post_weekday', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
spbCenter_pw_relation.columns = ['mean', 'count', 'median']
spbCenter_pw_relation.plot(y = 'mean')
spbCenter_pw_relation

**ВЫВОД: в обьявлениях, опубликовнных в субботу график показал всокую стоимости квадратного метра жилья, по сравнению с остальными днями. Также прослеживается снижение в среду и четверг. На общей выборке субботний пик также прослеживается**

In [ ]:
spbCenter_pm_relation = spbCenter_data.pivot_table(index = 'post_month', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
spbCenter_pm_relation.columns = ['mean', 'count', 'median']
spbCenter_pm_relation.plot(y = 'mean')
spbCenter_pm_relation

**ВЫВОДЫ: апрель получился самым дорогим для покупки жилья в центре. Снижения в маеи октябре. В целов в общей ыборке болеен стабильная картина по росту и снижению**

In [ ]:
spbCenter_py_relation = spbCenter_data.pivot_table(index = 'post_year', values = 'price_per_sqmeters', aggfunc = ['mean', 'count', 'median'])
spbCenter_py_relation.columns = ['mean', 'count', 'median']
spbCenter_py_relation.plot(y = 'mean')
spbCenter_py_relation

**ВЫВОД: снижение до 2019 года, как и в общем случае.**

### Шаг 5. Общий вывод

Стояла задача научиться определять рыночную стоимость объектов недвижимости.
Для этого необходимо установить параметры для построения автоматизированной системы, которая будет отслеживать аномалии и мошенническую деятельность.

Предобработка данных включала в себя предварительный анализ данных, работу с пропущенными значениями значениями и типами данных, а также проверку на дубликаты, чистку данных от артефактов и выбивающихся значний.  
Часть отсутствующих данных была восстановлена логическим путем и с использованием имеющихся данных. 
 В целом, критических замечаний по сборке и обработке данных нет. Основные причины отсутствующих данных  - пользователь. Возможно есть необходимость добавлять всплывающие подсказки для некоторых граф (например is_epartment), для пояснения пользователю какие данные должна содержать графа и для чего. Для некоторых граф, таких как удаленность от центра, близость аэропорта, парка и пр. - рассмотреть возможность автоматического определения и заполнения растояний на основе данных гис.
 
Далее были проведены расчеты и добавлены в таблицу небходимые для анализа параметры, проведена необходимая категоризация.

Рассчитаны и добавлены к имеющимся данным:
#цена кв. метра
#день недели месяц и год рубликации объявления
#Этаж квартиры разбит на категории(первый последний, другой)
#Отношение жилой и общей площади
#Отношение площади кухни к общей


На следующем этапе, для определения зависимостей были изучены следующие параметры: площадь, цeна, число комнат, высота потолков. 

В выборку попали площади от 12 до 900 кв. м. Среднее значение 60. Большая часть продаваемых квартир - это квартиры 40-60 кв. м Стоимость активно продаваемого жилья 3,4 - 6,8 млн. руб. В основном это 1,2,3х комнатные квартиры, преобладают 1,2х комнатные. Высота потолков в основном 2,5 - 2,7 м., есть 3х метровые. 

Провели анализ время продажи квартиры:

максимум квартиру продавали 4 с лишним года. В среднем квартира продается за пол года. Ходя медиана около 4 месяцев. В основной массе квартиры продаются в первые 90 дней. Максимум квартиру продавали 4 с лишним года . В среднем квартира продается за пол года. Медианное значение около 4 месяцев. Странные пики на 45 и 60 могут быть системными (например снятие обьявление с публикации через опр период)

Эти данные необходимо использовать для формирования логики отслеживания выбивающихся и подозрительных значений.
Так же, для того, что бы будущая система могла адекватно оценивать аномальную и мошенническую деятельность, были установлены зависимости цены от различных параметров продаваемого жилья ( площадь, число комнат, удаленность от центра, этаж, на кот. расположена кваритра).

Были установлены следующие зависимости:

**Цена зависит от площади жилья** 
с жильем до 100 м кв. цена колеблется примерно от 25000 до 200000 т. С увеличением площади жилья выше 100м кв., прослеживается незначительный рост цены за кв. метр. С увеличением площади жилья разброс цен сужается, предположительно завышать стоимость жилья на значительные площади сложнее, поэтому цена за кв. метр стремится к средним значениям.

**Есть зависимость стоимости картиры от количества комнат:**
Стоимость квартиры логично повышается с увеличением числа комнат. Но квартиры от 7 до 10 комнат судя по графику стоят почти одинаково. Стоимость квадратного метра жилья снижается до 3х комнатных квартир. Самая низкая стоимость квадратного метра жилья у 3х комнатных квартир. от 3х комнат и выше стоимость квадратного метра снова растет.

**Установлена зависимость цены от удаленности от центра:**¶
ближе к центру города определенно увеличивается сегмент дорогого жилья, но нижняя граница цен растет незначительно, т.е в центре есть некий сегмент недвижимости по стандартным для города ценам - вопрос, что это за жилье и какого уровня.

**Есть четкая зависимость цены от этажа, на кот. расположена кв.**

самый дешевый первый этаж, посдледний деше стоит дешевле чем другие, но дороже чем первый

**Проведена проверка на наличие зависимости цены от даты размещения: дня, месяца, года:**
 Период в год скорее отражает общую рыночную ситуацию. Периоды в месяц и неделя четкоой зависимости не показывают.
¶
рекордные цены по выборке в 2014 году - скорее всего это было спровоцированно валютным кризисом 2014-2015 г, далее все пошло на спад и к 2016 цена нормализовалась. в 2018 - заметное снижение
цена циклически снижается - повышается по кварталам, начиная с начала года. Пик в сентябре нарушает картину - возможно, какие то события подхлестнули спрос!
зависимость не прослеживается. Единственное, что можно увидеть - повышение цены в субботу

**Были оперделены минимум и максимум цены. А также населенные пункты с самой низкой и самой высокой стоимостью жилья¶**

Самые дорогие квартиры в Санкт-Петербурге и Пушкине, самые дшевые в Выборге!

**Следующим этапом был выделен и проанализирован сегмент квартир в центре города Санкт Петербург** 


В центре зависимость стоимости квадратного метра жилья от этажности сохраняется такая же как и по остальным районам первый - самый дешевый, последний подороже, чемпервый, остальные - в пользуются спросом и стоят дороже.
В центре зависимость цены от количества комнат незначительная.
Изменений цены по всему центру не наблюдается, все ровненько
В обьявлениях, опубликовнных в субботу график показал всокую стоимости квадратного метра жилья, по сравнению с остальными днями. Также прослеживается снижение в среду и четверг. На общей выборке субботний пик также прослеживается
Апрель получился самым дорогим для покупки жилья в центре. Снижения в маеи октябре. В целов в общей ыборке болеен стабильная картина по росту и снижению
Снижение до 2019 года, как и в общем случае.
